<a href="https://colab.research.google.com/github/ArshiAbolghasemi/AI-UT/blob/main/neural_network/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network(CNN)

A Convolutional Neural Network (CNN) is a type of deep learning model specifically designed for processing structured grid data like images. It consists of layers that apply convolution operations, pooling, and fully connected layers to automatically and adaptively learn spatial hierarchies of features from input images.

# Imports and Settings

## Imports

In [3]:
import os

from google.colab import drive

## Settings

In [5]:
DRIVE_MOUNTED_PATH=os.path.join(os.getcwd(), 'drive/')
TWITTER_SUICIDAL_DATASET_PATH=os.path.join(DRIVE_MOUNTED_PATH, 'MyDrive/AI-UT/neural_network/')

## Mounting to Google Drive

In [4]:
drive.mount(DRIVE_MOUNTED_PATH)

Mounted at /content/drive/
